# Vérification dynamique des types
Python 3.6 permet d'ajouter (de manière optionnelle) des annotations de type à notre programme.

In [1]:
def add(x: int, y: int) -> int:
    return x + y

Un outil comme [mypy](http://mypy-lang.org) permet d'analyser notre programme pour vérifier sa cohérence (analyse statique).

Par contre, les types ne seront pas vérifiés dynamiquement par CPython à l'exécution.

In [2]:
add(2, 3)

5

In [3]:
add("bonjour", " le monde")  # pas de problème !

'bonjour le monde'

On explore différentes manières de vérifier les types à l'exécution.

On peut commencer par vérifier le type des paramètres au début de la fonction :

In [4]:
def add(x: int, y: int) -> int:
    if not isinstance(x, int):
        raise ValueError(f"x est de type '{x.__class__.__name__}', 'int' attendu")
    if not isinstance(y, int):
        raise ValueError(f"y est de type '{y.__class__.__name__}', 'int' attendu")
    return x + y

In [5]:
add(2, 3)

5

In [6]:
add("bonjour", " le monde")

ValueError: x est de type 'str', 'int' attendu

C'est un petit peu répétitif, donc on peut factoriser avec une fonction :

In [7]:
def check_arg_type(name, value, expected):
    if not isinstance(value, expected):
        raise ValueError(f"{name} est de type '{value.__class__.__name__}', '{expected.__name__}' attendu")

def add(x: int, y: int) -> int:
    check_arg_type("x", x, int)
    check_arg_type("y", y, int)
    return x + y

In [8]:
add("bonjour", " le monde")

ValueError: x est de type 'str', 'int' attendu

Mais ça reste redondant avec les annotations de type : on dit deux fois la même chose.

Heureusement, ces annotations sont rattachées par Python à la fonction, ce qui nous permet de les inspecter :

In [9]:
add.__annotations__

{'x': int, 'y': int, 'return': int}

On peut ainsi écrire un décorateur qui va automatiquement vérifier que les types des paramètres correspondent aux annotations.

(Note : pour simplifier le problème, on ne considère ici que le cas des parmètres passés par mots-clés.)

In [10]:
def check_types(func):
    def wrapper(*args, **kwargs):
        for name, type_ in func.__annotations__.items():
            if name == "return":
                continue
            if not isinstance(kwargs[name], type_):
                raise ValueError(f"{name} est de type '{kwargs[name].__class__.__name__}', '{type_.__name__}' attendu")
        return func(*args, **kwargs)
    return wrapper


@check_types
def add(x : int, y : int) -> int:
    return x + y

In [11]:
add(x=2, y=3)

5

In [12]:
add(x="bonjour", y=" le monde")

ValueError: x est de type 'str', 'int' attendu

## Contrats

Une autre manière d'aborder la vérification des types des paramètres est de les considérer comme un cas particulier de **préconditions**, dans une approche qu'on appelle [programmation par contrat](https://fr.wikipedia.org/wiki/Programmation_par_contrat).

Un langage comme [Eiffel](https://fr.wikipedia.org/wiki/Eiffel_(langage)) inclut nativement ce genre de constructions, mais on peut les ajouter à Python au moyen de décorateurs.

Un exemple de bibliothèque qui implémente ce genre de choses est [PyContracts](https://pypi.org/project/PyContracts/), qui permet d'écrire des choses de ce genre :

In [13]:
!pip install PyContracts

In [14]:
from contracts import contract

@contract
def add(a: 'int', b: 'int,>0') -> 'int':
    return a + b

In [15]:
add(2, 3)

5

In [16]:
add(2, -5)

ContractNotRespected: Breach for argument 'b' to add().
Condition -5 > 0 not respected
checking: >0       for value: Instance of <class 'int'>: -5   
checking: int,>0   for value: Instance of <class 'int'>: -5   
Variables bound in inner context:


In [17]:
add("bonjour", "le monde")

ContractNotRespected: Breach for argument 'a' to add().
Could not satisfy any of the 3 clauses in Int|np_scalar_int|np_scalar,array(int).
 ---- Clause #1:   Int
 | Expected type 'int', got <class 'str'>.
 | checking: Int   for value: Instance of <class 'str'>: 'bonjour'   
 | Variables bound in inner context:
 | 
 ---- Clause #2:   np_scalar_int
 | Could not satisfy any of the 4 clauses in np_int8|np_int16|np_int32|np_int64.
 |  ---- Clause #1:   np_int8
 |  | Expected type 'int8', got <class 'str'>.
 |  | checking: np_int8   for value: Instance of <class 'str'>: 'bonjour'   
 |  | Variables bound in inner context:
 |  | 
 |  ---- Clause #2:   np_int16
 |  | Expected type 'int16', got <class 'str'>.
 |  | checking: np_int16   for value: Instance of <class 'str'>: 'bonjour'   
 |  | Variables bound in inner context:
 |  | 
 |  ---- Clause #3:   np_int32
 |  | Expected type 'int32', got <class 'str'>.
 |  | checking: np_int32   for value: Instance of <class 'str'>: 'bonjour'   
 |  | Var

## Pandas

In [18]:
import pandas as pd

Lorsque l'on traite des tableaux de données avec [Pandas](https://pandas.pydata.org), il arrive que les données qu'on reçoit en entrée ne soient pas du type attendu.

Parfois, on peut remplacer une donnée manquante (`NaN`) par une valeur par défaut avec [fillna](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html).

Parfois, on veut filtrer les lignes concernées, et ne garder que celles où telle colonne n'est pas vide.

Par exemple, cherchons à récupérer les adresses e-mail des sénateurs :

In [19]:
df = pd.read_csv(
        "https://data.senat.fr/data/senateurs/ODSEN_GENERAL.csv",
        sep=",",
        encoding="cp1252",
        skiprows=range(18),
    )

In [20]:
df.shape

(1738, 16)

Conservons seulement les colonnes pertinentes :

In [21]:
df = df[["Prénom usuel", "Nom usuel", "Courrier électronique"]]
df.head()

,Prénom usuel,Nom usuel,Courrier électronique
0,François,Abadie,Non public
1,Patrick,Abate,p.abate@senat.fr
2,Mohamed,Abdellatif,NaN
3,Nicolas,About,n.about@senat.fr
4,Youssef,Achour,NaN


On voit que certaines valeurs sont manquantes (`NaN`), on peut donc les filtrer :

In [22]:
df = df[df["Courrier électronique"].notnull()]
df.head()

,Prénom usuel,Nom usuel,Courrier électronique
0,François,Abadie,Non public
1,Patrick,Abate,p.abate@senat.fr
3,Nicolas,About,n.about@senat.fr
6,Philippe,Adnot,p.adnot@senat.fr
9,Leila,Aïchi,l.aichi@senat.fr


On voit que certaines valeurs sont "Non public", on peut les filtrer aussi :

In [23]:
df = df[df["Courrier électronique"] != "Non public"]
df

,Prénom usuel,Nom usuel,Courrier électronique
1,Patrick,Abate,p.abate@senat.fr
3,Nicolas,About,n.about@senat.fr
6,Philippe,Adnot,p.adnot@senat.fr
9,Leila,Aïchi,l.aichi@senat.fr
12,Jean-Paul,Alduy,jp.alduy@senat.fr
17,Pascal,Allizard,p.allizard@senat.fr
20,Jacqueline,Alquier,j.alquier@senat.fr
25,Michel,Amiel,m.amiel@senat.fr
26,Jean-Paul,Amoudry,jp.amoudry@senat.fr
28,Michèle,André,m.andre@senat.fr


Lorsque l'on crée un _pipeline_ de traitement de données, il peut être utile de vérifier que les données sont cohérentes (taille, format, valeurs...).

Pour cela, on peut utiliser la bibliothèque [engarde](https://github.com/TomAugspurger/engarde).